# Converting COPC to Potree Format

This notebook demonstrates how to convert CALIPSO COPC files to Potree format for web-based visualization.

## Why Convert to Potree?

- **Octree structure**: Efficient hierarchical LOD (Level of Detail) for large datasets
- **HTTP Range requests**: Load only visible parts of the data
- **Fast rendering**: Progressive loading and spatial indexing
- **Web-compatible**: Works directly in browsers without full file download

## The Challenge

CALIPSO COPC files are typically in **EPSG:3857 (Web Mercator)** with coordinates in meters, but PotreeConverter expects **EPSG:4326 (WGS84)** with coordinates in degrees.

**Solution**: Reproject using PDAL first, then convert to Potree.

## Prerequisites

### 1. Install PDAL (Point Data Abstraction Library)

```bash
# Using conda (recommended)
conda install -c conda-forge pdal python-pdal

# Or using pip
pip install pdal
```

### 2. Install PotreeConverter

Download from: https://github.com/potree/PotreeConverter/releases

Or compile from source:
```bash
git clone https://github.com/potree/PotreeConverter.git
cd PotreeConverter
mkdir build && cd build
cmake ..
make
```

### 3. Verify installations

In [ ]:
import subprocess
import sys

# Check PDAL
try:
    result = subprocess.run(['/opt/anaconda3/envs/pdal/bin/pdal', '--version'], 
                          capture_output=True, text=True)
    print(f"✅ PDAL installed: {result.stdout.strip()}")
except FileNotFoundError:
    print("❌ PDAL not found! Please install PDAL.")

# Check PotreeConverter
potree_path = './PotreeConverter/build/PotreeConverter'
try:
    result = subprocess.run([potree_path, '--help'], 
                          capture_output=True, text=True)
    print(f"✅ PotreeConverter found at: {potree_path}")
except FileNotFoundError:
    print(f"❌ PotreeConverter not found at {potree_path}")

## Step 1: Inspect Input COPC File

First, let's examine the coordinate system and bounds of our input COPC file.

In [ ]:
import json

# Path to your COPC file
input_copc = "output/CAL_LID_L1-Standard-V4-51.2023-06-30T16-44-43ZD.copc.laz"

# Get COPC metadata using PDAL
pdal_bin = '/opt/anaconda3/envs/pdal/bin/pdal'
result = subprocess.run(
    [pdal_bin, 'info', input_copc],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    info = json.loads(result.stdout)
    
    print("📊 COPC File Information:")
    print(f"  File: {input_copc}")
    
    if 'stats' in info:
        stats = info['stats']
        bbox = stats.get('bbox', {})
        native = bbox.get('native', {})
        
        print(f"  Points: {stats.get('statistic', [{}])[0].get('count', 'unknown'):,}")
        print(f"\n  Native Bounds (EPSG:3857 - Web Mercator):")
        print(f"    X: {native.get('minx', 0):,.2f} to {native.get('maxx', 0):,.2f} meters")
        print(f"    Y: {native.get('miny', 0):,.2f} to {native.get('maxy', 0):,.2f} meters")
        print(f"    Z: {native.get('minz', 0):,.2f} to {native.get('maxz', 0):,.2f} meters")
    
    # Check SRS
    if 'metadata' in info:
        srs = info['metadata'].get('srs', {}).get('compoundwkt', 'Not found')
        if 'EPSG:3857' in srs or 'Web_Mercator' in srs:
            print(f"\n  ✅ Confirmed: EPSG:3857 (Web Mercator) - needs reprojection")
        else:
            print(f"\n  ℹ️  SRS: {srs[:100]}...")
else:
    print(f"❌ Error reading COPC file: {result.stderr}")

## Step 2: Reproject COPC to WGS84

We'll use PDAL to reproject from EPSG:3857 (Web Mercator in meters) to EPSG:4326 (WGS84 in degrees).

### Why Reprojection is Critical:

- **EPSG:3857**: Uses meters, suitable for web maps but distorts at high latitudes
- **EPSG:4326**: Uses degrees (latitude/longitude), standard for geographic data
- **Scale matters**: We use `0.0000001` for lat/lon (≈1cm precision) and `0.001` for altitude (1mm precision)

In [ ]:
import tempfile
import os
from pathlib import Path

def reproject_copc_to_las(input_copc: str, output_las: str):
    """
    Reproject COPC from EPSG:3857 to EPSG:4326 using PDAL
    """
    print(f"📍 Reprojecting {Path(input_copc).name}...")
    print(f"   From: EPSG:3857 (Web Mercator, meters)")
    print(f"   To:   EPSG:4326 (WGS84, degrees)")
    
    # PDAL pipeline configuration
    pipeline = {
        "pipeline": [
            {
                "type": "readers.copc",
                "filename": input_copc
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:3857",
                "out_srs": "EPSG:4326"
            },
            {
                "type": "writers.las",
                "filename": output_las,
                # Critical: Use high precision for decimal degrees
                "scale_x": "0.0000001",  # ~1cm precision at equator
                "scale_y": "0.0000001",  # ~1cm precision
                "scale_z": "0.001",      # 1mm altitude precision
                "offset_x": "auto",
                "offset_y": "auto",
                "offset_z": "auto",
                "a_srs": "EPSG:4326"
            }
        ]
    }
    
    # Write pipeline to temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
        json.dump(pipeline, f, indent=2)
        pipeline_file = f.name
    
    try:
        # Run PDAL
        pdal_bin = '/opt/anaconda3/envs/pdal/bin/pdal'
        result = subprocess.run(
            [pdal_bin, 'pipeline', pipeline_file],
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✅ Reprojection complete: {output_las}")
        
        # Show output file size
        size_mb = os.path.getsize(output_las) / (1024 * 1024)
        print(f"   Output size: {size_mb:.1f} MB")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ PDAL reprojection failed:")
        print(f"  {e.stderr}")
        return False
    finally:
        os.unlink(pipeline_file)

# Create temporary LAS file
temp_las = tempfile.NamedTemporaryFile(suffix='.las', delete=False).name

# Run reprojection
success = reproject_copc_to_las(input_copc, temp_las)

if success:
    print(f"\n🎯 Intermediate LAS file created: {temp_las}")
    print(f"   This file is now in EPSG:4326 (WGS84) and ready for Potree conversion")

## Step 3: Convert LAS to Potree Format

Now we'll convert the reprojected LAS file to Potree's octree format.

### Potree Conversion Parameters:

- `--projection`: Specifies the coordinate system (WGS84 in this case)
- `--overwrite`: Replace existing output
- `--generate-page index`: Create a web viewer HTML page
- Output structure:
  - `metadata.json`: Bounding box, point count, hierarchy info
  - `hierarchy.bin`: Octree node structure
  - `octree.bin`: Point data organized in octree order

In [ ]:
def convert_las_to_potree(input_las: str, output_dir: str):
    """
    Convert LAS file to Potree format using PotreeConverter
    """
    print(f"🌲 Converting to Potree format...")
    print(f"   Input:  {input_las}")
    print(f"   Output: {output_dir}")
    
    # Create output directory
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # PotreeConverter command
    potree_converter = './PotreeConverter/build/PotreeConverter'
    cmd = [
        potree_converter,
        input_las,
        '-o', output_dir,
        '--overwrite',
        '--generate-page', 'index',
        # Specify WGS84 projection (PROJ.4 format)
        '--projection', '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'
    ]
    
    print(f"\nRunning PotreeConverter...")
    print(f"Command: {' '.join(cmd)}\n")
    
    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✅ Potree conversion complete!")
        print(f"\nPotreeConverter output:")
        print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ PotreeConverter failed:")
        print(f"  stdout: {e.stdout}")
        print(f"  stderr: {e.stderr}")
        return False
    except FileNotFoundError:
        print("❌ PotreeConverter not found!")
        print("Please install from: https://github.com/potree/PotreeConverter")
        return False

# Define output directory
output_potree_dir = "public/potree_data_test/test_conversion"

# Run conversion
if success:  # Only if reprojection succeeded
    conversion_success = convert_las_to_potree(temp_las, output_potree_dir)
else:
    print("⚠️  Skipping Potree conversion due to reprojection failure")

## Step 4: Verify Potree Metadata

Let's inspect the generated Potree metadata to ensure the conversion was successful.

In [ ]:
def verify_potree_metadata(output_dir: str):
    """
    Verify the generated Potree metadata has correct bounds
    """
    metadata_path = Path(output_dir) / "metadata.json"
    
    if not metadata_path.exists():
        print(f"❌ metadata.json not found at {metadata_path}")
        return False
    
    with open(metadata_path) as f:
        metadata = json.load(f)
    
    print("\n📊 Potree Metadata Verification:")
    print("="*60)
    
    # Basic info
    print(f"\n  Version: {metadata.get('version', 'unknown')}")
    print(f"  Total Points: {metadata.get('points', 0):,}")
    print(f"  Hierarchy Depth: {metadata.get('hierarchy', {}).get('depth', 'unknown')}")
    print(f"  Spacing: {metadata.get('spacing', 'unknown')}")
    
    # Bounding box
    bbox = metadata.get('boundingBox', {})
    min_coords = bbox.get('min', [])
    max_coords = bbox.get('max', [])
    
    if len(min_coords) >= 3 and len(max_coords) >= 3:
        print(f"\n  Bounding Box (WGS84):")
        print(f"    Longitude: {min_coords[0]:>10.6f}° to {max_coords[0]:>10.6f}°")
        print(f"    Latitude:  {min_coords[1]:>10.6f}° to {max_coords[1]:>10.6f}°")
        print(f"    Altitude:  {min_coords[2]:>10.3f}  to {max_coords[2]:>10.3f} km")
        
        # Scale and offset
        scale = metadata.get('scale', [])
        offset = metadata.get('offset', [])
        if len(scale) >= 3:
            print(f"\n  Scale:  [{scale[0]}, {scale[1]}, {scale[2]}]")
        if len(offset) >= 3:
            print(f"  Offset: [{offset[0]:.6f}, {offset[1]:.6f}, {offset[2]:.3f}]")
        
        # Validation
        print("\n  Validation:")
        checks_passed = True
        
        if abs(min_coords[0]) > 180 or abs(max_coords[0]) > 180:
            print("    ❌ Longitude outside valid range [-180, 180]°")
            checks_passed = False
        else:
            print("    ✅ Longitude within valid range [-180, 180]°")
        
        if abs(min_coords[1]) > 90 or abs(max_coords[1]) > 90:
            print("    ❌ Latitude outside valid range [-90, 90]°")
            checks_passed = False
        else:
            print("    ✅ Latitude within valid range [-90, 90]°")
        
        if max_coords[2] > 100:
            print(f"    ⚠️  Altitude > 100 km (suspicious for CALIPSO)")
            checks_passed = False
        else:
            print(f"    ✅ Altitude within expected range for CALIPSO (0-40 km)")
        
        if checks_passed:
            print("\n  🎉 All validation checks passed!")
        else:
            print("\n  ⚠️  Some validation checks failed")
        
        return checks_passed
    else:
        print("  ❌ Could not read bounding box from metadata")
        return False

# Verify the conversion
if conversion_success:
    verify_potree_metadata(output_potree_dir)
    
    # List output files
    print("\n📁 Generated Files:")
    for file in sorted(Path(output_potree_dir).rglob('*')):
        if file.is_file():
            size_mb = file.stat().st_size / (1024 * 1024)
            print(f"  {file.relative_to(output_potree_dir):<40} {size_mb:>8.2f} MB")

## Step 5: Cleanup Temporary Files

In [ ]:
# Clean up temporary LAS file
if os.path.exists(temp_las):
    os.unlink(temp_las)
    print(f"🧹 Cleaned up temporary LAS file: {temp_las}")

## Batch Processing: Convert Multiple Files

To convert all COPC files in a directory:

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import glob

def process_single_file(input_file, output_base_dir):
    """
    Process a single COPC file: reproject and convert to Potree
    """
    filename = Path(input_file).stem
    output_dir = Path(output_base_dir) / filename
    
    print(f"\n{'='*60}")
    print(f"Processing: {filename}")
    print(f"{'='*60}")
    
    # Create temporary LAS file
    temp_las = tempfile.NamedTemporaryFile(suffix='.las', delete=False).name
    
    try:
        # Step 1: Reproject
        if not reproject_copc_to_las(input_file, temp_las):
            return False, filename, "Reprojection failed"
        
        # Step 2: Convert to Potree
        if not convert_las_to_potree(temp_las, str(output_dir)):
            return False, filename, "Potree conversion failed"
        
        # Step 3: Verify
        if not verify_potree_metadata(str(output_dir)):
            return False, filename, "Validation failed"
        
        return True, filename, "Success"
    
    finally:
        # Cleanup
        if os.path.exists(temp_las):
            os.unlink(temp_las)

def batch_convert(input_dir, output_dir, max_workers=2):
    """
    Convert all COPC files in a directory to Potree format
    
    Args:
        input_dir: Directory containing COPC files
        output_dir: Base directory for Potree output
        max_workers: Number of parallel processes (default: 2)
    """
    # Find all COPC files
    copc_files = glob.glob(f"{input_dir}/*.copc.laz")
    
    if not copc_files:
        print(f"❌ No COPC files found in {input_dir}")
        return
    
    print(f"\n🔍 Found {len(copc_files)} COPC files to process")
    print(f"⚙️  Using {max_workers} parallel workers\n")
    
    results = []
    
    # Process files in parallel
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_single_file, file, output_dir): file 
            for file in copc_files
        }
        
        for future in as_completed(futures):
            success, filename, message = future.result()
            results.append((success, filename, message))
            
            if success:
                print(f"✅ {filename}: {message}")
            else:
                print(f"❌ {filename}: {message}")
    
    # Summary
    print(f"\n{'='*60}")
    print("BATCH CONVERSION SUMMARY")
    print(f"{'='*60}")
    
    successful = sum(1 for s, _, _ in results if s)
    failed = len(results) - successful
    
    print(f"  Total files: {len(results)}")
    print(f"  ✅ Successful: {successful}")
    print(f"  ❌ Failed: {failed}")
    
    if failed > 0:
        print("\n  Failed files:")
        for success, filename, message in results:
            if not success:
                print(f"    - {filename}: {message}")

# Example usage (uncomment to run):
# batch_convert(
#     input_dir="output",
#     output_dir="public/potree_data",
#     max_workers=2
# )

## Understanding Potree Output Structure

After conversion, your output directory will contain:

```
output_directory/
├── metadata.json          # Point cloud metadata (bounds, point count, etc.)
├── hierarchy.bin          # Octree structure (node hierarchy)
├── octree.bin            # Point data (positions, colors, attributes)
└── index.html            # Potree web viewer (if --generate-page was used)
```

### metadata.json Structure:

```json
{
  "version": "2.0",
  "name": "pointcloud",
  "points": 35063762,
  "projection": "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
  "hierarchy": {
    "firstChunkSize": 10000,
    "stepSize": 1000,
    "depth": 8
  },
  "boundingBox": {
    "min": [-179.996, -55.045, 0.381],
    "max": [179.999, 81.670, 40.000]
  },
  "scale": [0.0000001, 0.0000001, 0.001],
  "offset": [-179.996, -55.045, 0.381],
  "spacing": 1.5,
  "attributes": [
    {"name": "POSITION_CARTESIAN", "size": 12},
    {"name": "intensity", "size": 2},
    {"name": "classification", "size": 1},
    {"name": "gps-time", "size": 8}
  ]
}
```

## Troubleshooting Common Issues

### 1. Invalid Coordinates After Conversion

**Symptoms:**
- Latitude > 90° or < -90°
- Longitude > 180° or < -180°
- Altitude > 100 km (for CALIPSO)

**Causes:**
- Input COPC is not in EPSG:3857
- Wrong scale/offset values
- PotreeConverter interpreted meters as degrees

**Solution:**
```python
# Check input projection first:
pdal_bin = '/opt/anaconda3/envs/pdal/bin/pdal'
result = subprocess.run([pdal_bin, 'info', 'input.copc.laz'], 
                       capture_output=True, text=True)
# Look for "srs" in output
```

### 2. PotreeConverter Not Found

**Solution:** Update the `potree_converter` path in the code:
```python
potree_converter = '/full/path/to/PotreeConverter'
```

### 3. Out of Memory

**Solution:** Increase spacing to reduce point density:
```bash
PotreeConverter input.las -o output/ --spacing 2.0
```

### 4. Slow Processing

**Solution:** Use parallel processing (see batch conversion above) or process smaller chunks

## Summary

**Conversion Pipeline:**

```
COPC (EPSG:3857)  →  PDAL  →  LAS (EPSG:4326)  →  PotreeConverter  →  Potree Format
Web Mercator         Reproject   WGS84              Octree             Ready for Web
Meters               Transform   Degrees            Structure          Visualization
```

**Key Takeaways:**

1. ✅ **Always reproject first** - PDAL handles coordinate transformations correctly
2. ✅ **Use high precision scales** - `0.0000001` for lat/lon ensures ~1cm accuracy
3. ✅ **Verify metadata** - Check bounds are within valid geographic ranges
4. ✅ **Batch processing** - Use parallel workers for multiple files
5. ✅ **Validate output** - Ensure coordinates make sense for your data

**Expected Results for CALIPSO:**
- Longitude: -180° to 180° ✅
- Latitude: -82° to 82° (CALIPSO orbit limits) ✅
- Altitude: 0 to 40 km ✅

## Next Steps

After converting your data to Potree format:

1. **Copy to web server**: Move the output directory to `public/potree_data/`
2. **Load in viewer**: Update your application to load the Potree data
3. **Optimize performance**: Adjust LOD (Level of Detail) parameters for your use case
4. **Add filters**: Implement spatial bounds, time range, and attribute filters

See the main application code in `src/utils/potreeLoader.ts` for implementation details.